In [1]:
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier
import torchmetrics
from pytorch_lightning.utilities.model_summary import ModelSummary
import shutil
import tempfile
import os

/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.9/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.9/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/Users

In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [3]:
def save_file_as_mlflow_artifact(file_content_string, file_name='model_summary.txt'):
    tempdir = tempfile.mkdtemp()
    try:
        summary_file = os.path.join(tempdir, file_name)
        with open(summary_file, "w") as f:
            f.write(file_content_string)

        mlflow.log_artifact(local_path=summary_file)
    finally:
        shutil.rmtree(tempdir)

In [4]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    batch_size=64
)

  0%|          | 0/22500 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

  0%|          | 0/2500 [00:00<?, ?ex/s]

/Users/richardkuodis/miniforge3/envs/mlops/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
datamodule.num_classes

2

In [6]:
# if we want to track multiple metrics simultaneously, define a list of metrics and pass it to the classifier metric parameter
list_of_metrics = [
    torchmetrics.Accuracy(),
    torchmetrics.F1Score(num_classes=datamodule.num_classes),
    torchmetrics.Precision(num_classes=datamodule.num_classes),
    torchmetrics.Recall(num_classes=datamodule.num_classes)
]

In [7]:
# uncomment the following line and comment out the immediate next line to use a list of metrics instead of a single metric during model training
classifier_model = TextClassifier(
    backbone="prajjwal1/bert-tiny",
    num_classes=datamodule.num_classes,
    metrics=list_of_metrics
)

# classifier_model = TextClassifier(
#     backbone="prajjwal1/bert-tiny",
#     num_classes=datamodule.num_classes,
#     metrics=torchmetrics.F1(datamodule.num_classes)
# )
trainer = flash.Trainer(
    max_epochs=3,
    gpus=torch.cuda.device_count()
)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [8]:
EXPERIMENT_NAME = "dl_model_chapter03_2"
mlflow.set_tracking_uri('http://localhost')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 4


In [9]:
MODEL_ARTIFACT_PATH = 'dl_model'
REGISTERED_MODEL_NAME = 'nlp_dl_model'
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test(datamodule=datamodule)

    # mlflow log metrics
    cur_metrics = trainer.callback_metrics
    # Cast metric value as  float before passing into logger.
    metrics = dict(map(lambda x: (x[0], float(x[1])), cur_metrics.items()))
    mlflow.log_metrics(metrics)

    # mlflow log parameters
    params = {"epochs": trainer.max_epochs}
    if hasattr(trainer, "optimizers"):
        optimizer = trainer.optimizers[0]
        params["optimizer_name"] = optimizer.__class__.__name__
    if hasattr(optimizer, "defaults"):
        params.update(optimizer.defaults)
    # add hyper-parameters used in this training
    params.update(classifier_model.hparams)
    mlflow.log_params(params)

    # log model summary as an artifact
    summary = ModelSummary(classifier_model, max_depth=-1)
    save_file_as_mlflow_artifact(str(summary))

    # log and register the trained model
    mlflow.pytorch.log_model(
        pytorch_model=classifier_model,
        artifact_path=MODEL_ARTIFACT_PATH,
        registered_model_name=REGISTERED_MODEL_NAME
    )


  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
Restoring states from the checkpoint path at /Users/richardkuodis/Documents/Development/practical_mlflow/Practical-Deep-Learning-at-Scale-with-MLFlow/chapter03/lightning_logs/version_3/checkpoints/epoch=2-step=1053.ckpt
Loaded model weights from checkpoint at /Users/richardkuodis/Documents/Development/practical_mlflow/Practical-Deep-Learning-at-Scale-with-MLFlow/chapter03/lightning_logs/version_3/checkpoints/epoch=2-step=1053.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.647599995136261     │
│    test_cross_entropy     │    0.6334769129753113     │
│       test_f1score        │     0.647599995136261     │
│      test_precision       │     0.647599995136261     │
│        test_recall        │     0.647599995136261     │
└───────────────────────────┴───────────────────────────┘

Registered model 'nlp_dl_model' already exists. Creating a new version of this model...
2023/01/04 11:35:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nlp_dl_model, version 2
Created version '2' of model 'nlp_dl_model'.


In [10]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
                                               mlflow.get_run(run_id).info.lifecycle_stage))

run_id: f687480db90842f084062bf65fe243e6; lifecycle_stage: active


In [11]:
logged_model = f'runs:/{run_id}/{MODEL_ARTIFACT_PATH}'
# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)

# To bypass a lightning-flash's bug, we need to set the stage to test so a loaded model can be used to do prediction
model.trainer.state.stage = 'test'

predict_module = TextClassificationData.from_lists(
    predict_data=[
        "Best movie I have seen.",
        "What a movie!",
        "What a news!",
    ],
    batch_size=1
)
predictions = trainer.predict(classifier_model, datamodule=predict_module)
print(predictions)

Predicting: 351it [00:00, ?it/s]

[[tensor([0.0971, 0.1074])], [tensor([ 0.2551, -0.0560])], [tensor([ 0.2985, -0.1851])]]


In [12]:
# model_registry_version = mlflow.register_model(logged_model, "nlp_dl_model")
# print(f"Model name: {model_registry_version.name}")
# print(f"Model version: {model_registry_version.version}")